# Google Vision AI Python Data Processing
## Alexander O. Smith
## Created: August 15, 2021
The intention of this script is to create a function which takes Vision AI json output, parce them, cleans the data, and compiles them into a format that is easy for Python or R to do various memetic analyses on.

In [10]:
# Import Modules
import json, os, re
import pandas as pd


# Set Relevant Directories
#os.getcwd()
datadir = '/home/alexander0/Documents/Research/GoogleVisionAI/GoogleVisionAITestData/'

AttributeError: module 'os' has no attribute 'time'

In [6]:
with open(f'GoogleVisionAITestData/AndThatsAFact2.json') as f:
    db = json.load(f)

In [7]:
text = db['textAnnotations'][0]['description'] 
#gets OCR recovered text in the image.
#more data exists for each token's location if useful.
text

"we need to update\nthis meme because\nit's almost 8 months\nold now.\nAnd that's a fact\n"

Questions:
1. How much of the formatting is useful in general?
2. How do I reduce noise (such as watermarks, etc) which can justifiably be considered unimportant text?
3. How much does the location of the text matter?*

* It is interesting in the case of "And That's A Fact" meme to point out that "And that's a fact" is PewDiePie's response to some information on a television. That infomration is often in text format, but the text on the TV and the text of "And that's a fact" are in very different locations within the image in most cases. The meaning here might be to say "And that's a fact" is the character, PewDiePie, responding to an agentially distinguishable utterance. If this is important to the meme structure, then perhaps the only way to determine this is by spacial distance of text. If it is not, then we can bypass this concern.

In [4]:
spoof = db['safeSearchAnnotation']['spoof']
# Spoof according to Google Vision AI annotation is "The likelihood that an modification was made to the image's canonical version to make it appear funny or offensive."
# there are also other factors of whether it should be "safe search" which relates, I'm assuming, to Google's safe search criteria. Most of this is likely not as useful to me unless I want to exclude some results.
spoof
db['safeSearchAnnotation']

{'adult': 'VERY_UNLIKELY',
 'medical': 'VERY_UNLIKELY',
 'racy': 'VERY_UNLIKELY',
 'spoof': 'VERY_LIKELY',
 'violence': 'UNLIKELY'}

In [19]:
labels = []; score = []
for i in range(1, len(db['labelAnnotations'])):
    l = db['labelAnnotations'][i]['description']
    s = db['labelAnnotations'][i]['score']
    labels.append(l); score.append(s)
    image_labels = {'Label': labels, 'Score': score}

labels_df = pd.DataFrame(image_labels)
labels_df

,Label,Score
0,Hearing,0.861176
1,Audio equipment,0.836948
2,Gadget,0.832346
3,Beard,0.817370
4,Eyewear,0.817295
5,Electronic device,0.750747
6,Peripheral,0.742121
7,Personal protective equipment,0.719993
8,Fun,0.718744
9,Headphones,0.690645


In [8]:
# Import Modules
import json, os, re
import pandas as pd

def GoogleVision_parce(dir):
    # This is a loop that gathers all the required data from the json
    for file in os.listdir(dir): #dir should contain the json files
    # NOTE: if I connect this to mongo datatbase, I would instead have to specify the database with mongo 
    # credentials. I believe I can borrow this from other scripts Sam or Jeff built.
    
        if file.endswith(".json"):   #look for all files ending in .json
            f_short = re.sub('\.json', '', file) #make a variable to name the output dataframe
            #print(f_short) #debugging           

        # Open each JSON file as f in a loop
            with open(f'GoogleVisionAITestData/{file}') as f:
                #print(f'Loading {file}...') #debugging
                db = json.load(f) #load json file
                #print(f'{file} has loaded.')#debugging

                # gets the text and "spoof" likelihood
                text = db['textAnnotations'][0]['description'] 
                spoof = db['safeSearchAnnotation']['spoof']

                #begin building dataframe by making two lists.
                #labels defines the category; score contains the text, likelihood, or ratio score
                labels = ['spoof', 'text']; score = [spoof, text]
                # for loop gets all the label annotations, and appends them to labels
                for i in range(1, len(db['labelAnnotations'])):
                    # getting the label annotations and scores iteratively
                    l = db['labelAnnotations'][i]['description'] #label
                    s = db['labelAnnotations'][i]['score']       #numeric score for label
                    

                    # building the dataframe
                    labels.append(l); score.append(s) #appends label and score to list
                    image_labels = {'Label': labels, 'Score_Text': score} #makes a dict of data
                    globals()[f'{f_short}_df'] = pd.DataFrame(image_labels) #turns dict into a dataframe
                    # notice the naming convention for the dataframe is the shortened json name (without .json)
                    # this will likely need modification if we connect this to a larger dataset.
                    
# Set Relevant Directories
#os.getcwd() #current working directory
#os.chdir()  #change where you want output csv files to be put
datadir = '/home/alexander0/Documents/Research/GoogleVisionAI/GoogleVisionAITestData/' #dir of json files to parse.
                
# TO-DO: 
# create another function to connect to Google Vision AI API, & call it in the above function
# what is a good way to output the results to a csv? There are lots of ways to do this.
## 1. Each json/image output gets a csv, 2. Append a specified set of json outputs to a csv (deal with different row
## numbers for different images), 3. ???
# Check to see if there's any additional data when you pay for Google Vision AI.

# NOTES FOR FUTURE DEVELOPMENT:
# 1. Use the naming convention IMG_<#>.[png, jpg, etc], GVAI_<#>.json, & OUT_<#>.csv such that all the files can be 
# traced to their source image with # as the ID.
# 2. This #, could trace back to a tweet ID file, ID_<#>, however, tweet IDs may contain up to 4 images, so there 
# needs to be a convention to address this contextually. For example each IMG could be IMG_[1-n]_<#>.[extension]
# so long as this maps well with our API output from whatever platform.
# 3. Error handling needs to be addressed with greater numbers of images, with image deletions, etc.
# 4. I could generalize this script to deal with a variety of different social media APIs/Jeff's Mongo databases to 
# create a pipeline of data. However, before I do this, I would need to know I was getting all the necessary data from
# each JSON for this to be useful. But once that was made, it could be a script that runs from terminal on twitter 
# collections we have on Mongo, and could be appended back to the original tweet json files.


In [9]:
GoogleVision_parce(datadir)

Label: Eyelash; Score: 0.7797338
Label: Technology; Score: 0.7492561
Label: Electronic device; Score: 0.7433707
Label: Gadget; Score: 0.7080773
Label: T-shirt; Score: 0.7038516
Label: Event; Score: 0.6676291
Label: Multimedia; Score: 0.66351897
Label: Room; Score: 0.6627031
Label: Display device; Score: 0.6545849
Label: Smile; Score: 0.6451304
Label: Advertising; Score: 0.6364832
Label: Facial hair; Score: 0.6274806
Label: Screenshot; Score: 0.60069096
Label: Beard; Score: 0.5953405
Label: Media; Score: 0.58087295
Label: Brand; Score: 0.576205
Label: Job; Score: 0.56534356
Label: Photo caption; Score: 0.53738135
Label: Fun; Score: 0.5238801
Label: Hearing; Score: 0.8611759
Label: Audio equipment; Score: 0.8369484
Label: Gadget; Score: 0.83234584
Label: Beard; Score: 0.81737024
Label: Eyewear; Score: 0.81729543
Label: Electronic device; Score: 0.7507469
Label: Peripheral; Score: 0.7421208
Label: Personal protective equipment; Score: 0.71999335
Label: Fun; Score: 0.71874416
Label: Headph

In [3]:
AndThatsAFact2_df

,Label,Score_Text
0,spoof,VERY_LIKELY
1,text,we need to update\nthis meme because\nit's alm...
2,Hearing,0.861176
3,Audio equipment,0.836948
4,Gadget,0.832346
5,Beard,0.81737
6,Eyewear,0.817295
7,Electronic device,0.750747
8,Peripheral,0.742121
9,Personal protective equipment,0.719993


In [11]:
help(os)

Help on module os:

NAME
    os - OS routines for NT or Posix depending on what system we're on.

MODULE REFERENCE
    https://docs.python.org/3.7/library/os
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    This exports:
      - all functions from posix or nt, e.g. unlink, stat, etc.
      - os.path is either posixpath or ntpath
      - os.name is either 'posix' or 'nt'
      - os.curdir is a string representing the current directory (always '.')
      - os.pardir is a string representing the parent directory (always '..')
      - os.sep is the (or a most common) pathname separator ('/' or '\\')
      - os.extsep is the extension separator (always '.')
      - os.altsep is the alternate pathname se

SyntaxError: EOL while scanning string literal (<ipython-input-24-87ad5c6cace2>, line 1)